In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.1/StatisticalRethinking/zZGTK.ji for StatisticalRethinking [2d09df54-9d0f-5258-8220-54c2a3d4fbee]
└ @ Base loading.jl:1184
loaded

File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.030, 0.032, 0.032, 0.034) seconds, 0.13 seconds total
Sampling took (0.049, 0.053, 0.043, 0.045) seconds, 0.19 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -891  2.2e-02    1.00  -893  -891  -891  2.0e+03  1.1e+04  1.0e+00
accept_stat__   0.91  1.7e-03    0.12  0.66  0.95   1.0  4.6e+03  2.4e+04  1.0e+00
stepsize__      0.88  4.2e-02   0.059  0.83  0.88  0.97  2.0e+00  1.1e+01  3.1e+13
treedepth__      1.8  2.2e-02    0.52   1.0   2.0   3.0  5.7e+02  3.0e+03  1.0e+00
n_le

end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*